In [1]:
var('longitude latitude ellipsoidalHeight');
positionECEF(longitude,latitude,ellipsoidalHeight)=vector([
    (6378137.0 / (sqrt(1 - (0.081819190842622 * 0.081819190842622) * sin(latitude) * sin(latitude))) + ellipsoidalHeight) * cos(latitude) * cos(longitude),
    (6378137.0 / (sqrt(1 - (0.081819190842622 * 0.081819190842622) * sin(latitude) * sin(latitude))) + ellipsoidalHeight) * cos(latitude) * sin(longitude),
    (6378137.0 / (sqrt(1 - (0.081819190842622 * 0.081819190842622) * sin(latitude) * sin(latitude))) * (1 - (0.081819190842622 * 0.081819190842622)) + ellipsoidalHeight) * sin(latitude)
]);

In [2]:
var('depth');
r=vector([0,0,depth]);

In [3]:
var('x_offset y_offset z_offset');
offset=vector([x_offset,y_offset,z_offset]);

In [4]:
var('heading pitch roll');

In [5]:
C_ned_ecef=matrix([
[-cos(longitude)*sin(latitude),-sin(longitude),-cos(latitude)*cos(longitude)],
[-sin(latitude)*sin(longitude),cos(longitude),-cos(latitude)*sin(longitude)],
[cos(latitude),0,-sin(latitude)]
]);

In [6]:
C_imu_ned=matrix([
[cos(heading)*cos(pitch), cos(heading)*sin(pitch)*sin(roll)-cos(roll)*sin(heading), cos(heading)*cos(roll)*sin(pitch)+sin(roll)*sin(heading)],
[cos(pitch)*sin(heading), cos(heading)*cos(roll)+sin(pitch)*sin(roll)*sin(heading), sin(heading)*cos(roll)*sin(pitch)-cos(heading)*sin(roll)],
[-sin(pitch),  cos(pitch)*sin(roll),  cos(roll)*cos(pitch)]
]);

In [7]:
Georef = positionECEF(longitude,latitude,ellipsoidalHeight) + C_ned_ecef*C_imu_ned * (offset + r);

In [8]:
J=jacobian(Georef,(longitude,latitude,ellipsoidalHeight,depth,x_offset,y_offset,z_offset,heading,pitch,roll));

In [9]:
var('longitudeSigma latitudeSigma ellipsoidalHeightSigma depthSigma xOffsetSigma yOffsetSigma zOffsetSigma headingSigma pitchSigma rollSigma');
Covariance(longitudeSigma,latitudeSigma,ellipsoidalHeightSigma,depthSigma,xOffsetSigma,yOffsetSigma,zOffsetSigma,headingSigma,pitchSigma,rollSigma) = [
        [longitudeSigma*longitudeSigma,0,0,0,0,0,0,0,0,0],
        [0,latitudeSigma*latitudeSigma,0,0,0,0,0,0,0,0], 
        [0,0,ellipsoidalHeightSigma*ellipsoidalHeightSigma,0,0,0,0,0,0,0],
        [0,0,0,depthSigma*depthSigma,0,0,0,0,0,0],
        [0,0,0,0,xOffsetSigma*xOffsetSigma,0,0,0,0,0],
        [0,0,0,0,0,yOffsetSigma*yOffsetSigma,0,0,0,0],
        [0,0,0,0,0,0,zOffsetSigma*zOffsetSigma,0,0,0],
        [0,0,0,0,0,0,0,headingSigma*headingSigma,0,0],
        [0,0,0,0,0,0,0,0,pitchSigma*pitchSigma,0],
        [0,0,0,0,0,0,0,0,0,rollSigma*rollSigma],
]

In [10]:
TPU=J*Covariance*J.transpose()

In [11]:
print(TPU[0][0]) #sdLongitude

(longitudeSigma, latitudeSigma, ellipsoidalHeightSigma, depthSigma, xOffsetSigma, yOffsetSigma, zOffsetSigma, headingSigma, pitchSigma, rollSigma) |--> ellipsoidalHeightSigma^2*cos(latitude)^2*cos(longitude)^2 + (cos(latitude)*cos(longitude)*cos(pitch)*cos(roll) + (cos(heading)*cos(roll)*sin(pitch) + sin(heading)*sin(roll))*cos(longitude)*sin(latitude) + (cos(roll)*sin(heading)*sin(pitch) - cos(heading)*sin(roll))*sin(longitude))^2*depthSigma^2 + (((cos(roll)*sin(heading)*sin(pitch) - cos(heading)*sin(roll))*cos(longitude)*sin(latitude) - (cos(heading)*cos(roll)*sin(pitch) + sin(heading)*sin(roll))*sin(longitude))*(depth + z_offset) + (cos(longitude)*cos(pitch)*sin(heading)*sin(latitude) - cos(heading)*cos(pitch)*sin(longitude))*x_offset + ((sin(heading)*sin(pitch)*sin(roll) + cos(heading)*cos(roll))*cos(longitude)*sin(latitude) - (cos(heading)*sin(pitch)*sin(roll) - cos(roll)*sin(heading))*sin(longitude))*y_offset)^2*headingSigma^2 + ((ellipsoidalHeight + 6.37813700000000e6/sqrt(-0.00

In [12]:
print(TPU[1][1]) #sdLatitude

(longitudeSigma, latitudeSigma, ellipsoidalHeightSigma, depthSigma, xOffsetSigma, yOffsetSigma, zOffsetSigma, headingSigma, pitchSigma, rollSigma) |--> ellipsoidalHeightSigma^2*cos(latitude)^2*sin(longitude)^2 + (cos(latitude)*cos(pitch)*cos(roll)*sin(longitude) + (cos(heading)*cos(roll)*sin(pitch) + sin(heading)*sin(roll))*sin(latitude)*sin(longitude) - (cos(roll)*sin(heading)*sin(pitch) - cos(heading)*sin(roll))*cos(longitude))^2*depthSigma^2 + (((cos(roll)*sin(heading)*sin(pitch) - cos(heading)*sin(roll))*sin(latitude)*sin(longitude) + (cos(heading)*cos(roll)*sin(pitch) + sin(heading)*sin(roll))*cos(longitude))*(depth + z_offset) + (cos(pitch)*sin(heading)*sin(latitude)*sin(longitude) + cos(heading)*cos(longitude)*cos(pitch))*x_offset + ((sin(heading)*sin(pitch)*sin(roll) + cos(heading)*cos(roll))*sin(latitude)*sin(longitude) + (cos(heading)*sin(pitch)*sin(roll) - cos(roll)*sin(heading))*cos(longitude))*y_offset)^2*headingSigma^2 + ((ellipsoidalHeight + 6.37813700000000e6/sqrt(-0.00

In [13]:
print(TPU[2][2]) #sdHeight

(longitudeSigma, latitudeSigma, ellipsoidalHeightSigma, depthSigma, xOffsetSigma, yOffsetSigma, zOffsetSigma, headingSigma, pitchSigma, rollSigma) |--> (cos(pitch)*cos(roll)*sin(latitude) - (cos(heading)*cos(roll)*sin(pitch) + sin(heading)*sin(roll))*cos(latitude))^2*depthSigma^2 + (x_offset*cos(latitude)*cos(pitch)*sin(heading) + (cos(roll)*sin(heading)*sin(pitch) - cos(heading)*sin(roll))*(depth + z_offset)*cos(latitude) + (sin(heading)*sin(pitch)*sin(roll) + cos(heading)*cos(roll))*y_offset*cos(latitude))^2*headingSigma^2 + ((cos(latitude)*cos(pitch)*cos(roll) + (cos(heading)*cos(roll)*sin(pitch) + sin(heading)*sin(roll))*sin(latitude))*(depth + z_offset) + (cos(heading)*cos(pitch)*sin(latitude) - cos(latitude)*sin(pitch))*x_offset + (cos(latitude)*cos(pitch)*sin(roll) + (cos(heading)*sin(pitch)*sin(roll) - cos(roll)*sin(heading))*sin(latitude))*y_offset - (ellipsoidalHeight + 6.33543932729282e6/sqrt(-0.00669437999014140*sin(latitude)^2 + 1))*cos(latitude) - 42411.8382613839*cos(lat

In [14]:
TPUXTest=sqrt(TPU[0][0].subs(longitudeSigma=0.000001,latitudeSigma=0.000001,ellipsoidalHeightSigma=0.1,rollSigma=0.5,pitchSigma=0.5,headingSigma=1,longitude=-68.536125,latitude=48.443432,ellipsoidalHeight=-28,roll=45,pitch=45,heading=5,depth=10,x_offset=1,y_offset=2,z_offset=4,xOffsetSigma=0.00001,yOffsetSigma=0.00001,zOffsetSigma=0.00001,depthSigma=0.1))
print(TPUXTest.n())


15.9064753087618


In [15]:
TPUYTest=sqrt(TPU[1][1].subs(longitudeSigma=0.000001,latitudeSigma=0.000001,ellipsoidalHeightSigma=0.1,rollSigma=0.5,pitchSigma=0.5,headingSigma=1,longitude=-68.536125,latitude=48.443432,ellipsoidalHeight=-28,roll=45,pitch=45,heading=5,depth=10,x_offset=1,y_offset=2,z_offset=4,xOffsetSigma=0.00001,yOffsetSigma=0.00001,zOffsetSigma=0.00001,depthSigma=0.1))
print(TPUYTest.n())

4.25361902761076


In [16]:
TPUZTest=sqrt(TPU[2][2].subs(longitudeSigma=0.000001,latitudeSigma=0.000001,ellipsoidalHeightSigma=0.01,rollSigma=0.5,pitchSigma=0.5,headingSigma=1,longitude=-68.536125,latitude=48.443432,ellipsoidalHeight=-28,roll=2,pitch=3,heading=5,depth=10,x_offset=1,y_offset=2,z_offset=4,xOffsetSigma=0.00001,yOffsetSigma=0.00001,zOffsetSigma=0.00001,depthSigma=0.1))
print(TPUZTest.n())

6.35409615553769


In [18]:
print(TPU)


[                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                       